# This Jupyter Notebook comprises the code to build a Flask-based app for a Mock Financial System in which can create, edit, delete and store, both, users and their transactions, and, ultimately, send to any given user an email with an autogenerated summary of their transactions.


## The ```%%writefile``` magic command will generate separate files by dint of modularity and to leave the notebook available for testing, debugging and editting purposes. 

## Pythonic Email Delivery

### Using smtplib, TLS and Outlook ®, emails will be sent to a given account.
### The authenticaton credentials for the sender will be stored in a pickle file, which only has to be created once. 

## The email should include the following information:
### Total Balance(Credit - Debit movements), Average Credit, Average Debit and the number of transactions per month. 
### As well, a brief preamble, a signature with a logo and a subject must be included.
## The email from the previous ```trxn.csv``` file should look something like this:
![email](images/imagesemail_1try.png)

##  A .CSV file including all their transactions will be attached to the email; it will have the following fashion: 
![transactions](images/imagestrxn.png)

## Pickle File for authentication credentials

### Fill the credentials dictionary, run the code once, and then erase the entries.

### This code will create a ```.pkl``` file which will contain the mailing credentials. If the file already exists, then it will load it. 

### This way, sharing code is possible without revealing personal credentials;  ```.gitignore``` already includes this ```.pkl``` file, so it's not passed onto the GitHUB repo. 

In [2]:
import pickle
import os
if not os.path.exists('secret_credentials.pkl'):
    credentials={}
    credentials['Sender Email'] = "" # sender email account
    credentials['Sender Password'] = "" # sender email account password
    with open('secret_credentials.pkl','wb') as f:
        pickle.dump(credentials, f)
else:
    credentials=pickle.load(open('secret_credentials.pkl','rb'))

## Pythonic Email Delivery Infra

### The next cell contains three functions: ```df2Summary```, ```createTrxnsSummaryMessage```,```sendEmailTLS```

### ```df2Summary ``` function manages a SQL query into the relevant data description for the summary. 
### ```createTrxnsSummaryMessage``` creates the HTML styled content for the email with the results from ```df2Summary```.
### ```sendEmailTLS``` manages the email delivery with TLS level of security.

In [3]:
%%writefile email_functions.py
import smtplib, ssl, base64

from email import encoders
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase

import pandas as pd
import pickle
import os

    
def df2Summary(trxns):
    '''
        input =>    
            trxns_file: Pandas Data Frame
        output =>   
            trxns: Pandas DataFrame
            total_balance: Float
            avg_credit: Float
            avg_debit: Float
            monthly_sumarry: String
                
    '''
    #Total Balance
    total_balance = trxns.transaction.sum() # sum of transactions
    #Debit and Credit Averages
    trxns["DebitorCredit"] = trxns.transaction.map( lambda x: 'credit' if x > 0 else 'debit')
    avg_credit = trxns.loc[trxns.DebitorCredit == "credit", 'transaction'].mean()
    avg_debit = trxns.loc[trxns.DebitorCredit == "debit", 'transaction'].mean()
    #Monthly Summaries
    trxns['c'] = pd.to_datetime(trxns.created_at, format = "%Y-%m-%d")
    trxns['month'] = trxns.c.dt.strftime("%B")
    trx_per_month = trxns.month.value_counts()
    monthly_summary_email = ""
    monthly_summary = []
    #Create an html-formatted string to inject into the email body for only the months included
    for month, count in zip(trx_per_month.index, trx_per_month):
        monthly_summary_email += f"<br>Number of transactions in {month}: {count}</br>\n"
        monthly_summary.append([month, count])
    return trxns, total_balance, avg_credit, avg_debit, monthly_summary, monthly_summary_email

def createTrxnsSummaryMessage(total_balance, avg_credit, avg_debit, monthly_summary_email):
    '''
        Input =>
            total_balance: Float
            avg_credit: Float
            avg_debit: Float
            monthly_summary: String
        Output=>
            message => SMTP.MIMEMultipart Message Object
    '''
    
    message = MIMEMultipart("alternative")
    filename = "images/imageslogo_knot.png"
    image = base64.b64encode(open(filename, "rb").read())
    image_base64 = image.decode()

    
    html = f"""
    <html>
        <body>
            <div>
                <p>
                    Hello,<br>Herein is your Transactions Summary<br>
                </p>
            </div>
            <div>
                <table style="width:100%">
                      <tr>
                        <th>
                            <p>
                                <br>Total Balance: {total_balance}</br>
                                <br>Average Debit Amount: {avg_debit}</br>
                                <br>Average Credit Amount: {avg_credit}</br>
                            </p>
                        </th>
                        <th>
                            <p>
                                {monthly_summary_email}
                            </p>
                        </th>
                      </tr>
                </table>
            </div>  
            <div>
                <p>
                    <br>Engr. Raúl Armando Murga Garrido</br>
                    <br>B. Sc. Mechatronics Engineering</br>
                    <br><a href="https://www.linkedin.com/in/ra%C3%BAl-murga/">LinkedIn</a></br>
                    <br><a href="https://github.com/ramg93">GitHub</a></br>
                    <br><img align="left" width="150" height="150" src= "data:image/jpg;base64,{image_base64}"></br>
                </p>
            </div>
        </body>

    </html>
    """
    html_text = MIMEText(html, "html")
    message.attach(html_text)
    
    return message

def sendEmailTLS(credentials, receiver_email, subject, message, *filename):
    '''
        Input =>
            credentials: Dict
            receiver_email: String
            subject: String
            message: String}
        Output =>
            None: print either success or exception
    '''
    
    sender_email = credentials['Sender Email']
    password =credentials['Sender Password']
    
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = receiver_email
    
    if filename:

        with open(filename[0], "rb") as attachment:
    
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read()) 
        encoders.encode_base64(part)
        part.add_header(
            "Content-Disposition",
            f"attachment; filename= {filename[0]}",
        )
        message.attach(part)
    
    try:
        smtp = smtplib.SMTP("smtp-mail.outlook.com", 587)
        smtp.starttls()
        smtp.login(sender_email, password)
        smtp.sendmail(sender_email, receiver_email, message. as_string())
        smtp.quit()
        
        return f"email sent successfully to {receiver_email}"
    except Exception as e:
        return f"error sending email: {e}"

Overwriting email_functions.py


# Mock Financial System App 

## ```email_sender_app.py``` is the main app file. 

###  It has the calls from Flask, SQLAlchemy and the Database Migrations. 
### The SQL Dialect is sqlite 3



In [2]:
%%writefile email_sender_app.py
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret_key'
db_name = "esa.db"
app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{db_name}'

db = SQLAlchemy(app)
migrate = Migrate(app, db)

from routes import *

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

Overwriting email_sender_app.py


# ```models.py``` contains the SQL database models for Users and Transactions. 

## PythonicEmailSender.mwb and PythonicEmailSender.mwb.bak contain the database diagram for MySQL BenchWork

In [13]:
%%writefile models.py
from email_sender_app import db
from datetime import datetime

class User(db.Model):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True)
    created_at = db.Column(db.Date, nullable = False, default=datetime.utcnow())
    name = db.Column(db.String(255), nullable = False)
    lastname = db.Column(db.String(255), nullable = False)
    email = db.Column(db.String(255), nullable = False)
    # relationships 
    transactions = db.relationship('Transaction', back_populates='user', cascade="all,delete")# one2many

    def __repr__(self):
        return f'id = {self.id}: {self.name} {self.lastname}, {self.created_at}'


class Transaction(db.Model):
    __tablename__ = 'transaction'
    id = db.Column(db.Integer, primary_key=True)
    created_at = db.Column(db.Date, nullable = False, default=datetime.utcnow())
    transaction = db.Column(db.Integer)
    # relationships
    user_id = db.Column(db.Integer,  db.ForeignKey("user.id"))# many2one
    user = db.relationship("User", back_populates="transactions")
    
    def __repr__(self):
        return f'id = {self.id}: {self.transaction}, {self.created_at}'

Overwriting models.py


# ```routes.py``` contains, basically, all backend functions for each part of the app.

## it has comments separating the different parts of the app so that it's easy ti navigate.

In [4]:
%%writefile routes.py
from email_sender_app import app, db
from flask import render_template, url_for, flash, redirect

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

import models
from models import *
import forms

import pickle
import os
from email_functions import *

@app.route('/')
def base():
    return render_template('base.html')

# ***************************************************** Users **************************************************
@app.route('/users')
def users():
    users = models.User.query.all()
    return render_template('users.html', users=users)

@app.route('/add_user', methods=['GET', 'POST'])
def add_user():
    form = forms.AddUserForm()
    if form.validate_on_submit():
        user = models.User(name=form.name.data,
                           lastname=form.lastname.data,
                           email=form.email.data
                                )
        db.session.add(user)
        db.session.commit()
        flash('User added')
        return redirect(url_for('users'))
    return render_template('add_user.html', form=form)

@app.route('/edit_user/<int:user_id>', methods=['GET', 'POST'])
def edit_user(user_id):
    form = forms.AddUserForm()
    user = models.User.query.get(user_id)
    print(user)
    if user:
        if form.validate_on_submit():
            user.name = form.name.data
            user.lastname = form.lastname.data
            user.email = form. email.data

            db.session.commit()
            flash('User updated')
            return redirect(url_for('users'))
        
        form.name.data = user.name
        form.lastname.data = user.lastname
        form.email.data = user.email
        return render_template('edit_user.html', form=form, user_id=user_id)
    flash(f'User with id {user_id} does not exit')
    return redirect(url_for('users'))


@app.route('/delete_user/<int:user_id>', methods=['GET', 'POST'])
def delete_user(user_id):
    form = forms.DeleteForm()
    user = models.User.query.get(user_id)
    if user:
        if form.validate_on_submit():
            if form.submit.data:
                db.session.delete(user)
                db.session.commit()
                flash('User deleted')
            return redirect(url_for('users'))
        return render_template('delete_user.html', form=form, user_id=user_id)
    flash(f'User with id {user_id} does not exit')
    return redirect(url_for('users'))

# ***************************************************** Transactions **************************************************
@app.route('/transactions')
def transactions():
    transactions = models.Transaction.query.all()
    return render_template('transactions.html', transactions=transactions)

@app.route('/add_transaction/<int:user_id>', methods=['GET', 'POST'])
def add_transaction(user_id):
    form = forms.AddTransactionForm()
    if form.validate_on_submit():
        user = db.session.query(models.User).filter(models.User.id == user_id)[0]
        transaction = models.Transaction(transaction=form.transaction.data, 
                                user_id=user.id,
                                user=user
                                )
        db.session.add(transaction)
        db.session.commit()
        flash('Transaction added')
        return redirect(url_for('transactions_user', user_id = user_id))
    return render_template('add_transaction.html', form=form, user_id = user_id)

@app.route('/edit_transaction/<int:transaction_id>', methods=['GET', 'POST'])
def edit_transaction(transaction_id):
    form = forms.AddTransactionForm()
    transaction = models.Transaction.query.get(transaction_id)
    user_id = transaction.user_id
    print(transaction)
    if transaction:
        if form.validate_on_submit():
            transaction.transaction = form.transaction.data

            db.session.commit()
            flash('Transaction updated')
            return redirect(url_for('transactions_user', user_id = user_id))
        form.transaction.data = transaction.transaction
        return render_template('edit_transaction.html',
                               form=form,
                               transaction_id=transaction_id,
                               user_id = user_id)
    flash(f'Transaction with id {transaction_id} does not exit')
    return redirect(url_for('transactions_user', user_id = user_id))

@app.route('/delete_transaction/<int:transaction_id>', methods=['GET', 'POST'])
def delete_transaction(transaction_id):
    form = forms.DeleteForm()
    transaction = models.Transaction.query.get(transaction_id)
    user_id = transaction.user_id
    if transaction:
        if form.validate_on_submit():
            if form.submit.data:
                db.session.delete(transaction)
                db.session.commit()
                flash('Transaction deleted')
            return redirect(url_for('transactions_user',
                                    user_id = user_id))
        return render_template('delete_transaction.html',
                               form=form,
                               transaction_id=transaction_id,
                               user_id = user_id)
    flash(f'Transaction with id {transaction_id} does not exit')
    return redirect(url_for('transactions_user', user_id = user_id))

# ***************************************************** Transactions User *********************************************

@app.route('/transactions_user/<int:user_id>', methods=['GET', 'POST'])
def transactions_user(user_id):
    
    engine = create_engine("sqlite:///esa.db")
    
    Session = sessionmaker(bind = engine)
    session = Session()
    
    query = session.query(Transaction.transaction,
                     Transaction.created_at, Transaction.id).filter_by(
        user_id = user_id).statement

    df = pd.read_sql_query(query, engine)
    session.close()

    trxns, total_balance, avg_credit, avg_debit, monthly_summary, _ = df2Summary(df)
    
    transactions = models.Transaction.query.filter_by(user_id = user_id).all()
    user = models.User.query.get(user_id)
    name = user.name
    lastname = user.lastname
    print(trxns)
    
    return render_template('transaction_user.html',
                           name = name,
                           user_id = user_id,
                           lastname = lastname,
                           total_balance = total_balance,
                           avg_credit = avg_credit,
                           avg_debit = avg_debit,
                           monthly_summary = monthly_summary,
                           df = trxns,
                           transactions = transactions)

@app.route('/send_summary/<int:user_id>', methods=['GET', 'POST'])
def send_summary(user_id):
    user = models.User.query.get(user_id)
    credentials=pickle.load(open('secret_credentials.pkl','rb'))
    
    engine = create_engine("sqlite:///esa.db")
    
    Session = sessionmaker(bind = engine)
    session = Session()
    
    query = session.query(Transaction.transaction,
                     Transaction.created_at, Transaction.id).filter_by(
    user_id = user_id).statement

    df = pd.read_sql_query(query, engine)
    session.close()

    filename = f"Transactions_Summary_User_{user_id}.csv"
    df.to_csv(filename)
    print(filename)
    
    receiver_email = user.email
    trxns, total_balance, avg_credit, avg_debit, _, monthly_summary_email = df2Summary(df)
    message = createTrxnsSummaryMessage(total_balance, avg_credit, avg_debit, monthly_summary_email)
    response = sendEmailTLS(credentials, receiver_email, "Transactions Summary", message, filename)
    flash(response)
    os.remove(filename)
    return redirect(url_for('transactions_user', user_id = user_id))

Overwriting routes.py


# ```forms.py``` has the code to manage Flask's WTF. 

In [48]:
%%writefile forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, SelectField
from wtforms.validators import DataRequired, Email


class DeleteForm(FlaskForm):
    submit = SubmitField('Delete')


class AddUserForm(FlaskForm):
    name = StringField('Name', validators=[DataRequired()])
    lastname = StringField('Lastname', validators=[DataRequired()])
    email = StringField('email', validators=[DataRequired(), Email()])
    submit = SubmitField('Submit')
    

class AddTransactionForm(FlaskForm):
    transaction = StringField('Transaction', validators=[DataRequired()])
    submit = SubmitField('Submit')

Overwriting forms.py


# Templetes: HTML files

# Users: Display all users, add a new user, edit user and Delete user

In [36]:
%%writefile templates/users.html

{% extends "base.html" %}

{% block main %}
  {% for user in users %}
    <div class="card">
      <div class="card-body">
        <h3>{{ user.name }} {{ user.lastname }}</h3>
        <h4> {{ user.email }}</h4>
        <p>Added on {{ user.created_at }}</p>
        <a href="{{ url_for('edit_user', user_id=user.id) }}" class="btn btn-primary">Edit</a>
        <a href="{{ url_for('delete_user', user_id=user.id) }}" class="btn btn-danger">Delete</a>
        <a href="{{ url_for('transactions_user', user_id=user.id) }}" class="btn btn-secondary">Transactions</a>
      </div>
    </div>
  {% endfor %}
{% endblock %}

Overwriting templates/users.html


In [38]:
%%writefile templates/add_user.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('add_user') }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Add User</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('users') }}" class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
      <div class="col">
        <h5> Name </h5>
        {{ form.name(class="form-control") }}
      </div>
      <div class="col">
        <h5> Lastname </h5>
        {{ form.lastname(class="form-control") }}
      </div>
      <div class="col">
        <h5> Email </h5>
        {{ form.email(class="form-control") }}
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/add_user.html


In [12]:
%%writefile templates/edit_user.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('edit_user', user_id=user_id)  }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Add User</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('users') }}" class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
      <div class="col">
        <h5> Name </h5>
        {{ form.name(class="form-control") }}
      </div>
      <div class="col">
        <h5> Lastname </h5>
        {{ form.lastname(class="form-control") }}
      </div>
      <div class="col">
        <h5> Email </h5>
        {{ form.email(class="form-control") }}
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/edituser.html


In [2]:
%%writefile templates/delete_user.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('delete_user', user_id=user_id) }}" method="post">
    {{ form.csrf_token }}
    <p>Are you sure you want to delete the following user?</p>
    <div class="row">
      <div class="col">
        <h3>{{ user_id }}</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-danger") }}
        <a href="{{ url_for('users') }}" class="btn btn-primary">Cancel</a>
      </div>
    </div>
  </form>
</div>
{% endblock %}


Writing templates/deleteuser.html


# Transactions: Add a new Transactions, edit Transactions and Delete Transactions

In [99]:
%%writefile templates/add_transaction.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('add_transaction', user_id = user_id) }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Add Transaction</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('transactions_user', user_id=user_id) }}"class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
      <div class="col">
        <h5> Transaction </h5>
        {{ form.transaction(class="form-control") }}
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/add_transaction.html


In [100]:
%%writefile templates/edit_transaction.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('edit_transaction', transaction_id=transaction_id) }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Edit Transaction</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('transactions_user', user_id=user_id) }}" class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
        <div class="col">
          <h5> Transaction </h5>
          {{ form.transaction(class="form-control") }}
        </div>
      </div>
  </form>
</div>
{% endblock %}

Overwriting templates/edit_transaction.html


In [101]:
%%writefile templates/delete_transaction.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('delete_transaction', transaction_id=transaction_id) }}" method="post">
    {{ form.csrf_token }}
    <p>Are you sure you want to delete the following transaction?</p>
    <div class="row">
      <div class="col">
        <h3>{{ transaction_id }}</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-danger") }}
        <a href="{{ url_for('transactions_user', user_id=user_id) }}" class="btn btn-primary">Cancel</a>
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/delete_transaction.html


# Transactions Summary & Send Email

In [25]:
%%writefile templates/transaction_user.html

{% extends "base.html" %}

{% block main %}
<div class="card">
<div class="card-body">
    <body>
    
    <div class="row">
      <div class="col"> 
        <h2>
            Hello, {{ name }} {{ lastname }}
        </h2>  
        <br>
            <h3>
                Herein is your Transactions Summary
            </h3>
        <br>
        </div>
        <div class="col">
            <a href="{{ url_for('users') }}" class="btn btn-primary">Go Back</a>
            


            <a onclick="window.location.href='{{ url_for( 'send_summary', user_id = user_id) }}';" class="btn btn-Secondary">Send Summary</a>
            

        </div>
    </div>
    <div>
    <table style="width:100%">
        <tr>
            <th>
                <p>
                    <br>Total Balance: {{ total_balance }}</br>
                    <br>Average Debit Amount: {{ avg_debit }}</br>
                    <br>Average Credit Amount: {{ avg_credit }}</br>
                </p>
            </th>
            <th>
                <p>
                    
                    {% for month, count in monthly_summary %}
                        <br>Number of transactions in {{month}}: {{count}}</br>
                    {% endfor %}
                </p>
            </th>
        </tr>
    </table>
    </div> 
    </body>
</div>
</div>
<div class="card">
<div class="card-body">
    <div class="row">
        <div class="col"> 
            <h2> Transactions: </h2>
        </div>
        
        
        <div class="col"> 
            <a href="{{ url_for('add_transaction', user_id = user_id) }}" class="btn btn-primary">Add Transaction</a>
        </div>
    </div>
    
    {% for transaction in transactions %}
        <div class="card">
          <div class="card-body">
            <h3>{{ df.loc[df['id'] == transaction.id, 'DebitorCredit'].values[0].capitalize() }} Transaction of {{ transaction.transaction }}</h3>
            <h4></h4>
            <p>Transaction date:  {{ transaction.created_at }}</p>
            <a href="{{ url_for('edit_transaction', transaction_id=transaction.id) }}" class="btn btn-primary">Edit</a>
            <a href="{{ url_for('delete_transaction', transaction_id=transaction.id) }}" class="btn btn-danger">Delete</a>
          </div>
        </div>
        {% endfor %}
    
</div>
</div>
{% endblock %}

Overwriting templates/transaction_user.html


# Base

In [107]:
%%writefile templates/base.html

<!doctype html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=yes">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">


    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>
    <title>Pythonic Email Sender: Mock Financial Summary System</title>
  </head>
  <body>
    <div class="container-fluid">
      <h1 class="lead"> 
        <a href="{{ url_for('base') }}">Pythonic Email Sender: Mock Financial Summary System</a>
      </h1>
      <nav class="navbar navbar-expand-lg navbar-light bg-light">
        <ul class="navbar-nav">
          <li class="nav-item"><a href="{{ url_for('users') }}" class="nav-link">Users</a></li>
          <li class="nav-item"><a href="{{ url_for('add_user') }}" class="nav-link">Add User</a></li>
        </ul>
      </nav>
      <br>
      {% with messages = get_flashed_messages() %}
        {% if messages %}
          {% for message in messages %}
          <div class="alert alert-primary">
            {{ message }}
          </div>
          {% endfor %}
        {% endif %}
      {% endwith %}
      <br>
      {% block main %}{% endblock %}
    </div>
  </body>
</html>

Overwriting templates/base.html
